In [1]:
from datetime import datetime, timedelta, timezone
import threading
import time 

In [2]:
class Scheduler:
    """
    A class to represent a schedule for a job with a start and end date.
    
    Attributes:
    -----------
    stdate : str or datetime
        The start date and time of the schedule. If not provided, defaults to the current date and time.
    time_zone : str
        The timezone of the schedule. Defaults to the current timezone of the computer.
    
    Methods:
    --------
    add_job():
        Adds a job for running it 
    run_all():
        Runs all jobs with ot without threading
        
    Raises:
    -------
    ValueError:
        If Startdate is less than the current date.
    """

    def __init__(self, threading=False, startdate=None, time_zone=None):
        self.threads = threading
        self.stdate = startdate if startdate else datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        self.stdate = datetime.strptime(self.stdate, '%Y-%m-%d %H:%M:%S')
        self.jobs = []

        if time_zone is not None:
            self.time_zone = timezone(timedelta(hours=time_zone))
        else:
            # use the system's local timezone if no offset is provided
            self.time_zone = datetime.now().astimezone().tzinfo

        # checking if startdate is less than the current date
        if self.stdate.strftime('%Y-%m-%d') < datetime.now().strftime('%Y-%m-%d'):
            raise ValueError("Startdate cannot be less than the current date")
        
        # counting time for specified timezone 
        startdate_with_timezone = self.stdate.astimezone(self.time_zone)




############################# METHODS #############################
    
    ####### add_job ####### 
    def add_job(self, job):
        # if the job doesn't have a startdate, use the scheduler's startdate
        if job.startdate is None:
            job.startdate = self.stdate
            
        # if the job doesn't have a time_zone, use the scheduler's time_zone
        if job.time_zone is None:
            job.time_zone = self.time_zone
        # adding jobs 
        self.jobs.append(job)
        return self


    ####### run_all ####### 
    def run_all(self):
        
        # running jobs simultaneously when threads is true
        if self.threads:
            threads = []
            for job in self.jobs:
                # calling the Job run method for each thread
                thread = threading.Thread(target=job.run)
                threads.append(thread)
                
                # starting the execution of the thread
                thread.start()
            
            # ensure that run_all doesn't finish until all jobs have completed their execution.
            for thread in threads:
                thread.join()
        else:
            for job in self.jobs:
                # executing Job_s run method sequentially
                job.run()


In [6]:
# Example Job Class
class Job:
    def __init__(self, name, startdate =None, time_zone=None, duration=0):
        self.name = name
        self.startdate = startdate
        self.time_zone =time_zone
        self.duration = duration

        # setting time_zone 
        if time_zone is not None:
            self.time_zone = timezone(timedelta(hours=time_zone))
        



    
    # რეალური რანის ფუნქცია მაქვს დასაწერი !!!!!!!!!!!
    def run(self):
        # checking if self.startdate is a string
        if isinstance(self.startdate, str):
            # converting the string to a datetime object
            self.startdate = datetime.strptime(self.startdate, '%Y-%m-%d %H:%M:%S')
        
        # counting time for specified timezone 
        startdate_with_timezone = self.startdate.astimezone(self.time_zone)
        print(f' {self.name} started at {startdate_with_timezone}, timezone: {self.time_zone}')
        time.sleep(self.duration)
        print(f' {self.name} completed')



# Example Usage
scheduler = Scheduler(threading=True, startdate='2024-08-14 11:16:33')
scheduler.add_job(Job('Job 1', duration=1, time_zone=5))
scheduler.add_job(Job('Job 2', startdate='2024-08-14 11:16:33', duration=1))

scheduler.run_all()


 Job 1 started at 2024-08-14 12:16:33+05:00, timezone: UTC+05:00
 Job 2 started at 2024-08-14 11:16:33+04:00, timezone: Georgian Standard Time
 Job 1 completed Job 2 completed
